In [1]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Set the path to dataset
import os

home = '/content/drive/MyDrive/2ReadyForFineTuning/VGSI/'
os.chdir(home)
imroot = home+'Combined_Images'

In [3]:
# The data for VGSI is quite large (27 files with >50k samples in total), so fine-tuning may take too long
# You can configure how much data you want to use for fine-tuning/testing below
# Add 'VGSI_<id>_Format3' where id - 1 to 27 that you would like to use for fine-tuning/testing
prefix = ['VGSI_1_Format3'] #'VGSI_2_Format3', 'VGSI_13Format3'

In [4]:
# Prepare dataset in a desired format
import json
import ast
datalen = 0
prefix_combined = 'VGSI'
dumpdict = { "info": {}, "task_type": "Open-Ended", "data_type": "vgsi", "license": {}, "data_subtype": prefix_combined, "questions": [] }

combined_data = []
for prefix_itr in prefix:
  with open(home+prefix_itr+'.jsonl') as f:
    data=[]
    data = [json.loads(line) for line in f]
    combined_data.extend(data[0])

datalen = len(combined_data)
for ijson in combined_data:
  imlist = ijson['merged_image']
  anslist = ijson['answer_choices']
  passage = [str(i)+". "+ijson['passage'][i] for i in range(0,len(ijson['passage']))]
  try:
    assert len(anslist) == 4
  except:
    print(i)
  dumpdict['questions'].append( { "image_id": imroot+"/"+imlist, "question": ijson['question']+" "+"\n".join(passage), "question_id": ijson['File_Id'], "answer_choices": anslist, "answer_id": int(ijson['answer']) } )

with open('/content/'+prefix_combined+'.jsonl','w+') as w:
  w.write(json.dumps(dumpdict,indent=4))

print(dumpdict)

{'info': {}, 'task_type': 'Open-Ended', 'data_type': 'vgsi', 'license': {}, 'data_subtype': 'VGSI', 'questions': [{'image_id': '/content/drive/MyDrive/2ReadyForFineTuning/VGSI/Combined_Images/10000798.jpg', 'question': 'Choose the correct order of the events? 0. An ideal door would be one located in your laundry room or near your laundry facilities. Hanging your ironing board in this area of your home helps you stay organized by keeping your ironing board close to other laundry items. If you don’t have a laundry facility in your home, consider a place where you keep your laundry items, such as a closet that stores your detergent, laundry basket, etc. When hanging an ironing board over a door, the board can hang on either side. However, hanging the ironing board behind a door will present a more organized look as the board remains out of sight. While the over-the-door hanging method is one of the best ways to store your ironing board, there are many creative options out there.\n1. Once 

In [6]:
import json
f = open('/content/'+prefix_combined+'.jsonl','r')
data_questions = json.load(f)
questions = data_questions['questions']

print(data_questions.keys())
print("Number of questions:", len(questions))

dict_keys(['info', 'task_type', 'data_type', 'license', 'data_subtype', 'questions'])
Number of questions: 2000


In [7]:
# Install necessary libraries
!pip install torch
!pip install salesforce-lavis
!pip install transformers

In [8]:
# Import necessary libraries
import torch
from PIL import Image
from lavis.models import load_model_and_preprocess
from transformers import ViltProcessor, ViltForQuestionAnswering
import requests
from transformers import AutoProcessor, AutoModelForCausalLM
from huggingface_hub import hf_hub_download

/usr/local/lib/python3.10/dist-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/usr/local/lib/python3.10/dist-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


In [9]:
# Check if GPU is available (recommended but not necessary)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [10]:
# Load BLIP model finetuned on VQAv2
model_blip_vqa2, vis_processors_blip_vqa2, txt_processors_blip_vqa2 = load_model_and_preprocess(name="blip_vqa", model_type="vqav2", is_eval=True, device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

100%|██████████| 1.35G/1.35G [00:08<00:00, 162MB/s]
/usr/local/lib/python3.10/dist-packages/lavis/models/base_model.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  chec

# Batch Run

In [11]:
# Create a batch run script that executes BLIP on an item from the dataset
def batchrun(impath, question, answers):
  raw_image = Image.open(impath).convert("RGB")

  image_blip_vqa2 = vis_processors_blip_vqa2["eval"](raw_image).unsqueeze(0).to(device)
  question_blip_vqa2 = txt_processors_blip_vqa2["eval"](question)
  samples_blip_vqa2 = {"image": image_blip_vqa2, "text_input": question_blip_vqa2}
  bvqa2 = model_blip_vqa2.predict_answers(samples_blip_vqa2, answer_list=answers, inference_method="rank")[0]

  return bvqa2


In [13]:
# Call batch run script and write results as csv file
import csv
filprefix = "IPQ_BLIP_"+prefix_combined.upper()

with open(filprefix+'.csv', 'w+') as k:
  spamwriter = csv.writer(k)
  spamwriter.writerow(["qid","gt_ans","pred_ans_blip","correctness_blip"])
  for exid in range(0,datalen):
    print(exid)
    correctness_blip = 0
    #print(questions[exid]['image_id'], questions[exid]['question'], questions[exid]['answer_choices'])
    pred_ans_blip = batchrun(questions[exid]['image_id'], questions[exid]['question'], [str(i) for i in questions[exid]['answer_choices']])
    gtclass = questions[exid]['answer_choices'][int(questions[exid]['answer_id'])]
    if str(gtclass)==str(pred_ans_blip):
      correctness_blip = 1
    #print(exid,gtclass,pred_ans_blip,correctness_blip)
    spamwriter.writerow([exid,gtclass,pred_ans_blip,correctness_blip])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27